In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK") 
using JuMP
using GLPK 

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [11]:
# Construcción del modelo

modelo = Model(GLPK.Optimizer)

# Parámetros del Problema

meses = [1, 2, 3, 4, 5, 6]
demandas = [[1500, 3000, 2000, 4000, 2000, 2500], 
            [1300,  800,  800, 1000, 1100,  900],
            [2200, 1500, 2900, 1800, 1200, 2100],
            [1400, 1600, 1500, 1000, 1100, 1200]]
componentes = ["AX22-M1", "AX22-M2", "CX32-N1",  "CX32-N2"]
costosProduccion = [20, 25, 10, 15] 
costosBodega = [0.4, 0.5, 0.3, 0.3]
stockInicial = [10, 0, 0, 0]  
stockFinal = [50, 10, 10, 10]

# variable de decisión 
@variable(modelo, x[c in 1:length(componentes), m in meses] >= 0, Int)
@variable(modelo, y[c in 1:length(componentes), m in meses] >= 0, Int)

# Restricciones 

for c in 1:length(componentes)
  for m in 1:length(meses)
    @constraint(modelo, x[c, m] + stockInicial[c] >= demandas[c][m])
    if(m == 1)
      @constraint(modelo, y[c, m] == x[c, m] - demandas[c][m] +  stockInicial[c])
    end
    if(m > 1)
      @constraint(modelo, y[c, m] ==  x[c, m] - demandas[c][m] + y[c, m-1]) 
    end
    @constraint(modelo, y[c, 6] == stockFinal[c])
  end
end


#Función objetivo minimiza la suma de los costos de producción y almacenamiento

@objective(modelo, Min,
    sum(costosProduccion[c] * x[c, m] # costo de producción por la producción
          + costosBodega[c] * y[c, m] # por el costo de bodega por la cantidad en bodega
    for c in 1:length(componentes), m in meses)
)

optimize!(modelo)

println("Valor objetivo = ", objective_value(modelo))


for c in 1:length(componentes), m in 1:length(meses)
    println( "Mes: ", meses[m], " Componente: ", componentes[c], " Cantidad: ", value(x[c,m])," Cantidad en bodega: ", value(y[c,m]))
end



Valor objetivo = 682831.0
Mes: 1 Componente: AX22-M1 Cantidad: 1490.0 Cantidad en bodega: 0.0
Mes: 2 Componente: AX22-M1 Cantidad: 3000.0 Cantidad en bodega: 0.0
Mes: 3 Componente: AX22-M1 Cantidad: 2000.0 Cantidad en bodega: 0.0
Mes: 4 Componente: AX22-M1 Cantidad: 4000.0 Cantidad en bodega: 0.0
Mes: 5 Componente: AX22-M1 Cantidad: 2000.0 Cantidad en bodega: 0.0
Mes: 6 Componente: AX22-M1 Cantidad: 2550.0 Cantidad en bodega: 50.0
Mes: 1 Componente: AX22-M2 Cantidad: 1300.0 Cantidad en bodega: 0.0
Mes: 2 Componente: AX22-M2 Cantidad: 800.0 Cantidad en bodega: 0.0
Mes: 3 Componente: AX22-M2 Cantidad: 800.0 Cantidad en bodega: 0.0
Mes: 4 Componente: AX22-M2 Cantidad: 1000.0 Cantidad en bodega: 0.0
Mes: 5 Componente: AX22-M2 Cantidad: 1100.0 Cantidad en bodega: 0.0
Mes: 6 Componente: AX22-M2 Cantidad: 910.0 Cantidad en bodega: 10.0
Mes: 1 Componente: CX32-N1 Cantidad: 2200.0 Cantidad en bodega: 0.0
Mes: 2 Componente: CX32-N1 Cantidad: 1500.0 Cantidad en bodega: 0.0
Mes: 3 Componente: CX32